In [1]:
# import geemap,ee

Map = geemap.Map()

ee_shp = geemap.shp_to_ee(r"C:\Users\xiong\Documents\geemap\shp\boundary\boundary2.shp")

S1 = ee.ImageCollection("MODIS/061/MCD15A3H").select('Fpar').filterBounds(ee_shp.geometry())

# 将数据合并并转为list存放image
def map_m(i):
    # i is year list
    i = ee.Number(i)

    def func_dva(n):
        # n is 8 days list
        # 将时间从每年1月1号开始
        yr = ee.Date.fromYMD(i,1,1)
        # 将时间增加8天
        date = yr.advance(n, 'day')
        # 取时间戳
        day1 = yr.advance(n, 'day').millis()
        day2 = yr.advance(ee.Number(n).add(4),'day').millis()
        
#         # 每天合并为8天
#         day1 = yr.advance(n, 'day').millis()
#         day2 = yr.advance(ee.Number(n).add(1),'day').millis()
#         day3 = yr.advance(ee.Number(n).add(2),'day').millis()
#         day4 = yr.advance(ee.Number(n).add(3),'day').millis()
#         day5 = yr.advance(ee.Number(n).add(4),'day').millis()
#         day6 = yr.advance(ee.Number(n).add(5),'day').millis()
#         day7 = yr.advance(ee.Number(n).add(6),'day').millis()
#         day8 = yr.advance(ee.Number(n).add(7),'day').millis()
        
        # 筛选时间
        filter1 = ee.Filter.date(day1)
        filter2 = ee.Filter.date(day2)
        # 时间筛选出为image
        x1 = S1.filter(filter1)
        x2 = S1.filter(filter2)
        # 合并数据image
        col = x1.merge(x2)
        # 重命名image并求平均 
        fstr = ee.String('MCD_').cat(date.format('yyyy-MM-dd'))
        dat = col.mean().rename(fstr)
        return dat
    
    # 将一年分按照8天分割
    dats = ee.List.sequence(0, 361, 8).map(func_dva)

    return dats

# 获取时间ee.list
years = ee.List.sequence(2003, 2018, 1)
datalist = years.map(map_m)
# print(datalist.getInfo())

# 将image list数据转为imagecollection
image= ee.ImageCollection.fromImages(datalist.get(0))
for i in range(1, 16):
    d1 = ee.ImageCollection.fromImages(datalist.get(i))
    image = image.merge(d1)

print(image.getInfo())

NameError: name 'geemap' is not defined